In [1]:
import pandas as pd
import spacy
import yake

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_Columns', None)

In [3]:
data = pd.read_excel('World_Bank_Projects_downloaded_4_4_2022.xls')
data.head()

,id,regionname,countryname,projectstatusdisplay,project_name,pdo,impagency,cons_serv_reqd_ind,url,boardapprovaldate,closingdate,projectfinancialtype,curr_project_cost,curr_ibrd_commitment,curr_ida_commitment,curr_total_commitment,grantamt,borrower,lendinginstr,envassesmentcategorycode,esrc_ovrl_risk_rate,sector1,sector2,sector3,theme1,theme2
0,P252350,Africa East,Republic of Rwanda,Active,Rwanda Priority Skills for Growth (PSG),The Program Development Objective is to expand...,Ministry Of Education,NaN,http://projects.worldbank.org/P252350/null?lan...,2017-07-05T00:00:00Z,9/30/2023 12:00:00 AM,IDA,120000000.0,0.0,120000000.0,120000000.0,0.0,Ministry of Finance and Economic Planning,Program-for-Results Financing,NaN,NaN,Public Administration - Education,Tertiary Education,Workforce Development and Vocational Education,Education for the knowledge economy,NaN
1,P252338,East Asia and Pacific,Democratic Republic of Timor-Leste,Active,Road Climate Resilience Project Second Additio...,The project will deliver sustainable climate r...,"Ministry of Public Works, Transport, and Commu...",NaN,http://projects.worldbank.org/P252338/null?lan...,2017-04-27T00:00:00Z,NaN,NaN,39140000.0,0.0,35200000.0,35200000.0,0.0,Ministry of Finance,Investment Project Financing,B,NaN,Rural and Inter-Urban Roads,Public Administration - Transportation,NaN,NaN,NaN
2,P252337,Africa East,Republic of Kenya,Dropped,Kenya Devolution and Service Delivery,To improve citizen and government access to co...,Kenya National Bureau of Statistics,NaN,http://projects.worldbank.org/P252337/null?lan...,NaN,NaN,NaN,340000.0,0.0,0.0,0.0,340000.0,Kenya National Bureau of Statistics,Investment Project Financing,C,NaN,NaN,NaN,NaN,"Public expenditure, financial management and p...",Decentralization
3,P178946,Europe and Central Asia,Ukraine,Pipeline,Ukraine Emergency Public Service Support,The development objective is to sustain instit...,Ministry of Finance,TBD,http://projects.worldbank.org/P178946/null?lan...,NaN,NaN,NaN,950000000.0,950000000.0,0.0,950000000.0,0.0,Ukraine,Investment Project Financing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P178935,Middle East and North Africa,Republic of Iraq,Pipeline,Iraq Integrated Persistent Organic Pollutants ...,Improve the management and safe disposal of ta...,Ministry of Environment,TBD,http://projects.worldbank.org/P178935/null?lan...,NaN,NaN,NaN,108000000.0,0.0,0.0,0.0,13490000.0,Ministry of Finance,Investment Project Financing,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
categorical_col = data.select_dtypes("object")

In [5]:
categorical_col=categorical_col.drop(['id','url','cons_serv_reqd_ind'],axis=1)

In [6]:
categorical_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21445 entries, 0 to 21444
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   regionname                21442 non-null  object
 1   countryname               21442 non-null  object
 2   projectstatusdisplay      21441 non-null  object
 3   project_name              21444 non-null  object
 4   pdo                       11482 non-null  object
 5   impagency                 6540 non-null   object
 6   boardapprovaldate         18642 non-null  object
 7   closingdate               16495 non-null  object
 8   projectfinancialtype      12901 non-null  object
 9   borrower                  6309 non-null   object
 10  lendinginstr              21196 non-null  object
 11  envassesmentcategorycode  13082 non-null  object
 12  esrc_ovrl_risk_rate       1201 non-null   object
 13  sector1                   19530 non-null  object
 14  sector2               

In [7]:
categorical_col.isnull().sum()

regionname                      3
countryname                     3
projectstatusdisplay            4
project_name                    1
pdo                          9963
impagency                   14905
boardapprovaldate            2803
closingdate                  4950
projectfinancialtype         8544
borrower                    15136
lendinginstr                  249
envassesmentcategorycode     8363
esrc_ovrl_risk_rate         20244
sector1                      1915
sector2                     10355
sector3                     13930
theme1                       9190
theme2                      11550
dtype: int64

In [8]:
for i in categorical_col.columns:
    categorical_col[i] = categorical_col[i].fillna(categorical_col[i].mode()[0])

In [9]:
for i in categorical_col.columns:
    categorical_col[i] = categorical_col[i].str.lower()

In [10]:
from nltk.corpus import stopwords

In [11]:
stop_word = set(stopwords.words('english'))
for i in categorical_col.columns:
    categorical_col[i] = categorical_col[i].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_word))

In [12]:
for i in categorical_col.columns:
    categorical_col[i] = categorical_col[i].str.replace('\d+','')
    categorical_col[i] = categorical_col[i].str.replace(r'[!"\$%&\'()*+,\-.\/:;=#@?\[\\\]^_`{|}~]*','')

In [13]:
categorical_col.head()

,regionname,countryname,projectstatusdisplay,project_name,pdo,impagency,boardapprovaldate,closingdate,projectfinancialtype,borrower,lendinginstr,envassesmentcategorycode,esrc_ovrl_risk_rate,sector1,sector2,sector3,theme1,theme2
0,africa east,republic rwanda,active,rwanda priority skills growth psg,program development objective expand opportuni...,ministry education,tz,,ida,ministry finance economic planning,programforresults financing,b,substantial,public administration education,tertiary education,workforce development vocational education,education knowledge economy,rural services infrastructure
1,east asia pacific,democratic republic timorleste,active,road climate resilience project second additio...,project deliver sustainable climate resilient ...,ministry public works transport communications,tz,,,ministry finance,investment project financing,b,substantial,rural interurban roads,public administration transportation,central government central agencies,rural services infrastructure,rural services infrastructure
2,africa east,republic kenya,dropped,kenya devolution service delivery,improve citizen government access county data ...,kenya national bureau statistics,tz,,,kenya national bureau statistics,investment project financing,c,substantial,central government central agencies,central government central agencies,central government central agencies,public expenditure financial management procur...,decentralization
3,europe central asia,ukraine,pipeline,ukraine emergency public service support,development objective sustain institutional ci...,ministry finance,tz,,,ukraine,investment project financing,b,substantial,central government central agencies,central government central agencies,central government central agencies,rural services infrastructure,rural services infrastructure
4,middle east north africa,republic iraq,pipeline,iraq integrated persistent organic pollutants ...,improve management safe disposal targeted stoc...,ministry environment,tz,,,ministry finance,investment project financing,b,substantial,central government central agencies,central government central agencies,central government central agencies,rural services infrastructure,rural services infrastructure


In [14]:
import spacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [15]:
len(categorical_col)

21445

In [19]:
for i in range(len(categorical_col)):
    doc = nlp(categorical_col['pdo'][i])
    for token in doc.ents:
        if token.label_ == 'ORG':
            print(token.text, token.label_)
            print("Next Row")
            print("----"*35)

eastern nepal ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
el salvador´s national road ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
country´s national road ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
world bank gro government ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
country´s national road ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
pacific island ORG
Next Row
--------------------------------------------------------------------------------------

In [21]:
for i in range(len(categorical_col)):
    doc = nlp(categorical_col['sector1'][i])
    for token in doc.ents:
        if token.label_ == 'ORG':
            print(token.text, token.label_)
            print("Next Row")
            print("----"*35)

transportation ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
transportation ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
transportation ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
transportation ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
transportation ORG
Next Row
--------------------------------------------------------------------------------------------------------------------------------------------
transportation ORG
Next Row
-------------------------------------------------------------------------------------------------------------------------------